## Introduction

If you're reading this, you're probably familiar with [nbdev](https://nbdev.fast.ai/), a system for creating Python libraries from Jupyter notebooks. 

The only trouble I have with it is keeping track of dependencies, especially when you have multiple projects or expect other people to use your code.

In this post, I'll show you how to set up a new nbdev project using [Pipenv](https://pipenv.pypa.io/en/latest/), a tool for managing Python dependencies. This will allow you to easily create a new project, install dependencies, and run your code without having to worry about setting up a virtual environment.

View source code on GitHub [here](https://github.com/ddobrinskiy/blog/tree/master/posts/2022/12/nbdev_pipenv/src)


## Some background on Pipenv

If you're not familiar with Pipenv, it's a tool for managing Python dependencies. It's similar to [virtualenv](https://virtualenv.pypa.io/en/latest/), [conda](https://docs.conda.io/en/latest/) or [poetry](https://python-poetry.org/), but it's a bit more user-friendly.

Setting up a new project with Pipenv is easy. You just need to run `pipenv install` in the project directory. This will create a new virtual environment and install all the dependencies listed in the `Pipfile`.

If you want to add a new dependency, you can run `pipenv install <package_name>`. This will install the package and add it to the `Pipfile`, keeping track of the version you installed.

If you want to install a specific version of a package, you can run `pipenv install <package_name>==<version>`. This will install the package and add it to the `Pipfile` with the specified version.


Example Pipfile:

```{.toml filename="Pipfile"}
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
requests = "==2.28"
pandas = "*"

[dev-packages]
black = {version = "==22.12.0", extras = ["jupyter"]}
pre-commit = "==2.20"
ipykernel = "*"

[requires]
python_version = "3.10"

```

By default in nbdev you have to manually set all your dependencies in the `settings.ini` file, which are then parsed in `setup.py`. This is fine if you only have a few dependencies, but it can get messy if you have a lot of them.

We will be exporting the dependencies from the `Pipfile` to the `setup.cfg` file to override the default behavior of `setup.py`

## Step 1: generate `setup.cfg` from `Pipfile`

tbd

## Step 2: remove nbdev defaults

Comment out or remove dependencies from `settings.ini` (`requirements` and `dev_requirements`)

```{.python filename="settings.ini"}
...
### Optional ###
# requirements = fastcore pandas
# dev_requirements =  nbdev nbconvert nbdev_template
```

Now remove the `install_requires` and `extras_require` lines from `setup.py`, otherwise they will overwrite our `setup.cfg` config


```{.python filename="setup.py"}
setuptools.setup(
    ...
    include_package_data = True,
    # install_requires = requirements,
    # extras_require={ 'dev': dev_requirements },
    dependency_links = cfg.get('dep_links','').split(),
    ...
    )

```

## Step 3: install the package

test that everything works on a fresh environment

```{.bash}
pip install -e .
```


if you want to install dev packages as well, run

```{.bash}
pip install -e '.[dev]'
```

## Extra reading:

- [nbdev docs](https://nbdev.fast.ai/)
- [Pipenv docs](https://pipenv.pypa.io/en/latest/)
- [setuptools docs](https://setuptools.pypa.io/en/latest/userguide/quickstart.html)